# Introducción  

En Python, se puede interactuar con bases de datos relacionales de dos maneras principales:  

## Usando módulos que implementan DBAPI  

DBAPI es un estándar en Python que define una interfaz uniforme para interactuar con bases de datos relacionales, independientemente del sistema de base de datos utilizado.  

Algunos de los módulos más comunes que implementan DBAPI son:  
- `sqlite3`: Para bases de datos **SQLite** (incluido en la biblioteca estándar de Python, sin necesidad de instalación adicional).  
- `PyMySQL`: Para conectar con **MySQL**.  
- `psycopg2`: Para conectar con **PostgreSQL**.  

Este enfoque es el más básico, ya que el programador **escribe las consultas SQL manualmente y maneja las transacciones directamente**.  

**En este ejercicio, utilizaremos este enfoque (DBAPI) para interactuar con SQLite y MySQL.**  

## Usando Object-Relational Mapping (ORM)  

Es una técnica que permite mapear tablas de bases de datos relacionales a clases de objetos en lenguajes de programación orientados a objetos. En lugar de escribir consultas SQL manualmente, puedes manipular los datos de la base de datos utilizando clases y objetos Python.  

Un ejemplo de ORM en Python es **SQLAlchemy**, que permite trabajar con bases de datos de forma más abstracta.  


- **Si necesitas mayor rendimiento, usa DBAPI (`sqlite3`, `pymysql`, `psycopg2`).**
- **Si prefieres trabajar con objetos y una mayor abstracción, usa SQLAlchemy.**

Cada opción tiene su uso dependiendo del contexto del proyecto. 

# Carga Incremental de Datos desde SQLite a MySQL

En este ejercicio, desarrollaremos un sistema para realizar cargas incrementales de datos desde una base de datos SQLite hacia un **Operational Data Store (ODS)** en MySQL. 

## Contexto

La empresa **Zprinter** opera varias tiendas, cada una registrando pedidos en su propia base de datos SQLite. Para centralizar la información y facilitar la toma de decisiones estratégicas, consolidaremos estos datos en un ODS basado en MySQL. 

### Instrucciones:
1. Ejecutar los scripts de inicialización para poblar las bases de datos SQLite y MySQL.
2. Obtener la última fecha de actualización desde MySQL.
3. Extraer solo los nuevos o modificados registros desde SQLite.
4. Insertar estos registros en MySQL para actualizar la tabla `orders`.

# Ejecutar los scripts de inicialización

## SQLite

Ejecutaremos el script que inicializa la base de datos **SQLite** en el archivo `database/zprinter.db`.
SQLite es una base de datos **ligera y embebida** que no requiere de servidor. Se almacena en un solo archivo y es ideal para aplicaciones que necesitan una base de datos simple, por ejemplo, para desarrollo o aplicaciones móviles.

### ¿Qué hace este script?  
1. **Crea la carpeta `database/`** (si no existe) y el archivo `zprinter.db`, que contendrá nuestra base de datos.  
2. **Elimina la tabla `orders` si ya existe** y la vuelve a crear con las siguientes columnas:  
   - `order_id`: Identificador único del pedido.  
   - `client_id`: Identificador del cliente.  
   - `status`: Estado del pedido (`Pendiente de pago`, `En proceso`, etc.).  
   - `last_updated`: Fecha y hora de la última actualización.  
3. **Inserta pedidos de ejemplo**, incluyendo algunos que ya existen en MySQL y otros nuevos que se sincronizarán más adelante.  

Este paso es fundamental, ya que nos aseguramos de tener datos en SQLite antes de realizar la carga incremental hacia MySQL.

In [2]:
import os
import sqlite3

# Conectar a SQLite y crear la base de datos y tabla
os.makedirs("database", exist_ok=True)
conn_sqlite = sqlite3.connect("database/zprinter.db")
cursor_sqlite = conn_sqlite.cursor()

cursor_sqlite.execute("DROP TABLE IF EXISTS orders;")  # Elimina la tabla si ya existe
cursor_sqlite.execute("""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY,
    client_id INTEGER,
    status TEXT,
    last_updated TEXT
);
""")

# Insertar más registros que en MySQL
pedidos_iniciales = [
    (1, 100, "Pendiente de pago", "2024-04-01 10:00:00"),  # Ya existe en MySQL
    (2, 101, "En proceso", "2024-04-02 11:30:00"),        # Ya existe en MySQL
    (3, 102, "Enviado", "2024-04-03 12:45:00"),          # Nuevo (no está en MySQL)
    (4, 103, "Enviado", "2024-04-04 09:20:00"),          # Nuevo
    (5, 104, "En proceso", "2024-04-05 16:15:00"),       # Nuevo
    (6, 105, "Cancelado", "2024-04-06 14:10:00"),        # Nuevo
    (7, 106, "Pendiente de pago", "2024-04-07 18:00:00") # Nuevo
]

cursor_sqlite.executemany(
    "INSERT INTO orders (order_id, client_id, status, last_updated) VALUES (?, ?, ?, ?);",
    pedidos_iniciales
)
conn_sqlite.commit()
cursor_sqlite.close()
conn_sqlite.close()

print("Base de datos SQLite creada.")

Base de datos SQLite creada.


## MySQL

Para configurar la base de datos MySQL y crear la tabla `orders`, podemos ejecutar el siguiente script SQL de varias maneras.  

### **Opción 1: Usando la terminal de MySQL**  
1. **Abrir MySQL desde la terminal**  
   - Si tienes **MySQL instalado localmente**, abre la terminal y ejecuta:  
     ```sh
     mysql -u root -p
     ```
     Luego, ingresa tu contraseña cuando se te solicite.  

   - Si usas **MySQL en Docker**, primero inicia el contenedor y accede:  
     ```sh
     docker exec -it <nombre_del_contenedor> mysql -u root -p
     ```
  
2. **Ejecutar el script SQL**  
   Una vez dentro del entorno de MySQL, copia y pega el siguiente código y presiona **Enter**:  

   ```sql
   CREATE DATABASE IF NOT EXISTS nter_python; 
   USE nter_python;

   DROP TABLE IF EXISTS orders;
   CREATE TABLE IF NOT EXISTS orders (
     order_id INT PRIMARY KEY,
     client_id INT,
     status VARCHAR(30),
     last_updated TIMESTAMP
   );

   INSERT INTO orders (order_id, client_id, status, last_updated)
   VALUES
   (1, 100, 'Pendiente de pago', '2024-04-01 10:00:00'),
   (2, 101, 'En proceso', '2024-04-02 11:30:00');
   ```

---

### **Opción 2: Usando MySQL Workbench**  

Si prefieres una interfaz gráfica, puedes utilizar **MySQL Workbench** para ejecutar el script:  

1. **Abrir MySQL Workbench** y conectar con el servidor.  
2. **Crear una nueva consulta** haciendo clic en **"File" → "New Query Tab"**.  
3. **Copiar y pegar el script SQL** en el editor de consultas.  
4. **Ejecutar el script** presionando el botón de "Run" (⚡) o con `Ctrl + Enter`.  


## Obtener la última fecha de actualización desde MySQL.

## Conexión con MySQL

### Cargar librerías y configurar variables de entorno

Para conectar con **MySQL**, utilizamos variables de entorno almacenadas en un archivo `.env`; es preferible a tener las credenciales escritas directamente en el código.

Usamos `dotenv` para cargarlas desde el entorno:

Para **MySQL** se obtienen el host, puerto, usuario, contraseña y base de datos.

Para **SQLite** definimos la ruta del archivo de la base de datos SQLite

In [3]:
# Importamos las librerías necesarias
import os  # Para manejar variables de entorno y rutas de archivos
import sqlite3  # Para conectarnos y trabajar con bases de datos SQLite
import pymysql  # Para conectarnos y trabajar con bases de datos MySQL
from dotenv import load_dotenv  # Para cargar variables de entorno desde un archivo .env

# Cargar las variables de entorno desde un archivo .env
load_dotenv()

# Obtener las credenciales de conexión a MySQL desde las variables de entorno
MYSQL_HOST = os.getenv("MYSQL_HOST")  # Dirección del servidor MySQL
MYSQL_PORT = int(os.getenv("MYSQL_PORT", 3306))  # Puerto de conexión a MySQL (por defecto 3306)
MYSQL_USER = os.getenv("MYSQL_USER")  # Usuario de MySQL
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")  # Contraseña de MySQL
MYSQL_DB = os.getenv("MYSQL_DB")  # Nombre de la base de datos en MySQL

# Definir la ruta del archivo de la base de datos SQLite
SQLITE_DB_PATH = "database/zprinter.db"  # Ubicación del archivo SQLite local


### Para conectarnos a MySQL desde Python utilizando `pymysql`:

1. Se intenta establecer una conexión a MySQL utilizando `pymysql.connect()`.
2. Se pasan los parámetros de conexión como `host`, `user`, `password`, `db` y `port`.
3. Si la conexión es exitosa, se muestra un mensaje de confirmación.
4. Si ocurre un error, se captura la excepción `MySQLError` y se muestra un mensaje de error.

In [5]:
try:
    conn_mysql = pymysql.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        db=MYSQL_DB,
        port=MYSQL_PORT
    )
    print("Conectado a MySQL correctamente")
except pymysql.MySQLError as e:
    print(f"Error al conectarse a MySQL: {e}")


Conectado a MySQL correctamente


## Obtener la última fecha de actualización en MySQL

1. **Se verifica la conexión** (`if conn_mysql`) antes de ejecutar la consulta.
2. **Se crea un cursor** para ejecutar comandos SQL en MySQL.
3. **Consulta SQL:**
   - `MAX(last_updated)`: Obtiene la última fecha de actualización de la tabla `orders`.
   - `COALESCE(..., '1900-01-01')`: Si la tabla está vacía, devuelve `'1900-01-01'` como valor predeterminado.
4. **Se ejecuta la consulta** y se obtiene el resultado con `fetchone()`, que devuelve una única fila como una tupla.
5. **`fetchone()` y `result[0]`**:
   - `fetchone()` devuelve una tupla con los valores de la fila obtenida de la consulta.
   - `result[0]` se usa para extraer el primer (y único) valor de la tupla, que en este caso es la última fecha de actualización.
6. **Se cierra el cursor** para liberar recursos.
7. **Se imprime la última fecha de actualización**, o si no hay registros, se asigna `'1900-01-01 00:00:00'` por defecto.

In [6]:
if conn_mysql:
    cursor_mysql = conn_mysql.cursor()
    mysql_query = """SELECT COALESCE(MAX(last_updated), '1900-01-01') FROM orders;"""
    cursor_mysql.execute(mysql_query)
    result = cursor_mysql.fetchone()
    cursor_mysql.close()
    
    if result:
        last_updated = result[0]  
        print("Última actualización en MySQL:", last_updated)
    else:
        print("No se encontraron registros en MySQL.")
        last_updated = "1900-01-01 00:00:00"


Última actualización en MySQL: 2024-04-02 11:30:00


## Conectar a SQLite y obtener datos nuevos

In [7]:
conn_sqlite = sqlite3.connect(SQLITE_DB_PATH)
cursor_sqlite = conn_sqlite.cursor()


- Se usa `sqlite3.connect` para conectarse a la base de datos SQLite.
- Se crea un cursor (`cursor_sqlite`) para ejecutar consultas en la base de datos.

In [8]:
sqlite_query = """SELECT * FROM Orders WHERE last_updated > ?;"""
cursor_sqlite.execute(sqlite_query, (last_updated,))
results = cursor_sqlite.fetchall()

- Se usa `last_updated` obtenido de MySQL como referencia para extraer solo los registros más recientes.
- `fetchall()` recupera todas las filas que cumplen la condición.

In [9]:
cursor_sqlite.close()
conn_sqlite.close()

Se cierra el cursor y la conexión para liberar recursos y evitar bloqueos en la base de datos.

## Insertar los datos nuevos en MySQL

In [10]:
# Este código toma los registros obtenidos previamente de SQLite y los inserta o actualiza en MySQL
if results:
    try:
        # Se crea un cursor para ejecutar operaciones SQL en la conexión MySQL
        cursor_mysql = conn_mysql.cursor()
        
        # Se define la consulta SQL para insertar nuevos registros o actualizar los existentes.
        # La consulta utiliza 'ON DUPLICATE KEY UPDATE' para:
        # - Insertar un nuevo registro si 'order_id' no existe.
        # - Actualizar 'client_id', 'status' y 'last_updated' si el registro ya existe.
        insert_query = """
            INSERT INTO orders (order_id, client_id, status, last_updated)
            VALUES (%s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE 
                client_id=VALUES(client_id),
                status=VALUES(status),
                last_updated=VALUES(last_updated)
        """
        
        # Se ejecuta la consulta para cada elemento en 'results' utilizando 'executemany',
        # lo que permite procesar múltiples registros en una sola llamada.
        cursor_mysql.executemany(insert_query, results)
        
        # Se confirman los cambios en la base de datos
        conn_mysql.commit()
        print("Datos sincronizados con MySQL correctamente.")
    
    # Se captura cualquier excepción específica de pymysql y se muestra un mensaje de error.
    except pymysql.MySQLError as e:
        print(f"Error al insertar datos en MySQL: {e}")
    
    # El bloque 'finally' garantiza que, independientemente del resultado, se liberen los recursos.
    finally:
        cursor_mysql.close()
        conn_mysql.close()


Datos sincronizados con MySQL correctamente.
